In [1]:
import requests

url = "https://www.camara.cl/legislacion/proyectosdeley/leyes_promulgadas.aspx"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    with open("./web/leyes_promulgadas.html", "w", encoding="utf-8") as file:
        file.write(response.text)
    print("Página descargada con éxito.")
else:
    print(f"Error {response.status_code}: No se pudo descargar la página.")


Página descargada con éxito.


In [2]:
import pandas as pd
import re
from bs4 import BeautifulSoup



with open("./web/leyes_promulgadas.html", "r", encoding="utf-8") as file:
    soup = BeautifulSoup(file, "html.parser")

tabla = soup.find("table", class_="tabla")
datos = []

regex_numero_ley = re.compile(r"\d+\.\d+")

for fila in tabla.find_all("tr")[1:]:
    columnas = fila.find_all("td")
    
    if len(columnas) >= 6:
        numero_proyecto = columnas[0].text.strip()
        fecha_ingreso = columnas[1].text.strip()
        iniciativa = columnas[2].text.strip()
        proyecto_suma = columnas[3].text.strip()
        numero_ley = columnas[4].text.strip()
        fecha_publicacion = columnas[5].text.strip()

        numero_ley_match = regex_numero_ley.search(numero_ley)
        if numero_ley_match:
            numero_ley = int(numero_ley_match.group().replace(".", ""))  # Convertir a int
        else:
            numero_ley = None 

        datos.append([numero_proyecto, fecha_ingreso, iniciativa, proyecto_suma, numero_ley, fecha_publicacion])

df = pd.DataFrame(datos, columns=[
    "numero_proyecto", "fecha_ingreso", "iniciativa", "proyecto_suma", "numero_ley", "fecha_publicacion"
])

df = df.dropna(subset=['numero_ley'])

df["fecha_ingreso"] = pd.to_datetime(df["fecha_ingreso"], format="%d-%m-%Y", errors="coerce")
df["fecha_publicacion"] = pd.to_datetime(df["fecha_publicacion"], format="%d-%m-%Y", errors="coerce")
df['numero_ley'] = df['numero_ley'].astype(int)

print(df.dtypes)
print(df.head())

df.to_csv("./data/leyes_promulgadas.csv", index=False, encoding="utf-8")


numero_proyecto              object
fecha_ingreso        datetime64[ns]
iniciativa                   object
proyecto_suma                object
numero_ley                    int64
fecha_publicacion    datetime64[ns]
dtype: object
  numero_proyecto fecha_ingreso iniciativa  \
1        17286-05    2024-12-09    Mensaje   
2        16078-08    2023-07-10    Mensaje   
3        13062-24    2019-10-30     Moción   
4        15297-24    2022-08-18     Moción   
5        16598-15    2024-01-22    Mensaje   

                                       proyecto_suma  numero_ley  \
1  Otorga reajuste general de remuneraciones a la...       21724   
2  Modifica la Ley General de Servicios Eléctrico...       21721   
3  Autoriza a erigir un memorial en homenaje a lo...       21715   
4  Autoriza construir un monumento en homenaje a ...       21714   
5  Prohíbe la fabricación, comercialización, impo...       21720   

  fecha_publicacion  
1        2025-01-03  
2        2024-12-27  
3        2024-12-2

In [3]:
# from unidecode import unidecode

# palabras_clave = ['hidrocarburos', 'petroleo', 'gas', 'mineria', 'combustibles', 'energia', 'renovable', 'petrolera', 'medio ambiente']
# regex = r'\b(' + '|'.join(palabras_clave) + r')\b'

# df_filtrado = df[df['proyecto_suma'].apply(lambda x: bool(re.search(regex, unidecode(x), re.IGNORECASE)))]
# df_filtrado.reset_index(inplace=True, drop=True)
df_filtrado = df[df["fecha_ingreso"].dt.year == 2023]
df_filtrado.to_csv("./data/leyes_promulgadas_filtradas.csv", index=False, encoding="utf-8")


In [5]:
!pip install playwright
!playwright install chromium


In [6]:
!pip install nest_asyncio

In [9]:
lista_leyes_2023 = list(df_filtrado["numero_ley"])
numero_ley = lista_leyes_2023[0]
print(lista_leyes_2023)

[21721, 21717, 21707, 21703, 21699, 21697, 21694, 21684, 21679, 21677, 21672, 21670, 21673, 21668, 21674, 21665, 21666, 21662, 21655, 21657, 21656, 21658, 21648, 21644, 21644, 21650, 21654, 21653, 21652, 21637, 21645, 21645, 21645, 21645, 21639, 21638, 21647, 21640, 21636, 21629, 21623, 21622, 21626, 21631, 21628, 21617, 21620, 21625, 21619, 21616, 21618, 21614, 21596, 21611, 21613, 21604, 21593, 21594, 21609, 21602, 21599, 21598, 21590, 21588, 21581, 21586, 21586, 21583, 21583, 21584, 21579, 21578, 21572, 21573, 21574, 21569, 21569, 21550, 21540, 21543, 21700]


In [8]:
import asyncio
from playwright.async_api import async_playwright

async def scrape_page(numero_ley):
    url = f"https://www.bcn.cl/leychile/navegar?idLey={numero_ley}"
    print(url)

    async with async_playwright() as p:
        # Iniciar el navegador
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url)

        try:
            # Esperar a que el botón con role="button" esté disponible y hacer clic en él
            await page.wait_for_selector("span[role='button']", state="visible", timeout=5000)
            await page.click("span[role='button']")

            # Esperar a que el contenido dinámico se cargue (usamos un selector específico del contenido nuevo)
            await page.wait_for_selector("span[itemprop='accessibilitySummary']", state="visible", timeout=5000)
        except Exception as e:
            print(f"Error al hacer clic o cargar el contenido: {e}")
        finally:
            # Extraer el HTML después del clic
            html_content = await page.content()
            await browser.close()

    return html_content

# Ejecutar la función asíncrona
html_result = asyncio.run(scrape_page(numero_ley))

# Guardar el HTML en un archivo
with open("./web/leyes_resumen.html", "w", encoding="utf-8") as file:
    file.write(html_result)

print("HTML guardado correctamente.")

RuntimeError: asyncio.run() cannot be called from a running event loop

In [7]:
import asyncio
from playwright.async_api import async_playwright

async def scrape_page(numero_ley):
    url = f"https://www.bcn.cl/leychile/navegar?idLey={numero_ley}"

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url)

        try:
            # Hacer clic en el botón "Ver más >>"
            await page.click("span[role='button']")

            # Esperar a que el contenido dinámico se cargue
            await page.wait_for_selector("span[itemprop='accessibilitySummary']", state="visible", timeout=5000)

            # Extraer el HTML después del clic
            html_content = await page.content()
        except Exception as e:
            print(f"Error: {e}")
        finally:
            await browser.close()

    return html_content

# Obtener el bucle de eventos y ejecutar la tarea
loop = asyncio.get_event_loop()
html_result = loop.run_until_complete(scrape_page(numero_ley))

# Guardar el HTML en un archivo
with open("./web/leyes_resumen.html", "w", encoding="utf-8") as file:
    file.write(html_result)

print("HTML guardado correctamente.")

RuntimeError: This event loop is already running

In [13]:
from bs4 import BeautifulSoup

# Suponiendo que `html_content` contiene el HTML descargado con Playwright

def extraer_materias_resumen_ley(numero_ley, html_result):
    soup = BeautifulSoup(html_result, "html.parser")
    div_datos = soup.find("div", class_="datos")
    keywords_materias = div_datos.find_all("span", itemprop="keywords")
    materias = ""
    for span in keywords_materias:
        materias += span.text.strip()

    accessibility_summary = div_datos.find_all("span", itemprop="accessibilitySummary")
    summary_span = accessibility_summary[0]

    spans = summary_span.find_all("span")
    resumen_ley = spans[0].text.strip()

    return [numero_ley, materias, resumen_ley]

print(extraer_materias_resumen_ley(numero_ley, html_result))

NameError: name 'html_result' is not defined

In [ ]:
import time
list_resumen = []
for ley in lista_leyes_2023:
    print(ley)
    try:
        html_result = asyncio.run(scrape_page(ley))
        time.sleep(7)
        list_data = extraer_materias_resumen_ley(numero_ley, html_result)
        list_resumen.append(list_data)
    except:
        pass

df_resumen_ley = pd.DataFrame(list_resumen, columns=['numero_ley', 'materias', 'resumen'])
df_resumen_ley.to_csv("./data/leyes_resumen_2023.csv", index=False, encoding="utf-8")



21724
21721
21715
21714
21720
21722
21719
21719
21718
21717
21712
21710
21708
21716
21709
21706
21711
21707
21713
Error al hacer clic o cargar el contenido: Page.wait_for_selector: Timeout 5000ms exceeded.
Call log:
  - waiting for locator("span[role='button']") to be visible

21703
21698
21702
21699
21695
21697
21696
21705
21689
21704
21692
21701
21694
21686
21693
21690
21690
21690
21684
21683
21691
21688
21687
21685
21682
21678
21680
21679
21681
21676
21677
21672
21675
21670
21671
21673
21668
21674
21669
21664
21665
21667
21666
21660
21663
21662
21659
21655
21657
21656
21658
21651
21648
21644
21644
21650
21646
21654
21653
21652
21643
21637
21642
21642
21642
21649
Error al hacer clic o cargar el contenido: Page.wait_for_selector: Timeout 5000ms exceeded.
Call log:
  - waiting for locator("span[role='button']") to be visible

21641
21645
21645
21645
21645
21645
21639
21638
21647
21640
21635
21636
21634
Error al hacer clic o cargar el contenido: Page.wait_for_selector: Timeout 5000ms ex